# Reducing GPT Primer Files

Welcome! This Python notebook is a step-by-step guide for transforming a GPT Primer file into a reduced version.

### Background

A GPT Primer file, often used with OpenAI's language model, is a YML file that consists of metadata used to influence, constrain, and contextualize ChatGPT responses. This notebook will help you create a reduced version of such files, improving efficiency, reducing token usage, making for more consistent outputs.

## Function to import and primer from a file.

In [261]:
primer_name = 'data_science_tutor'

In [262]:
'''
Load the primer file from a source directory
'''
def get_primer(primer_name = 'data_science_tutor'):
    print('in get_primer()')
    
    input_fileType = '.yml'

    with open('./primers/' + primer_name + input_fileType, 'r') as f:
        primer = f.read()
        # print(primer)
        return primer

primer = get_primer()
toWrite = primer


in get_primer()


## Functions to manipulate the primer text

All functions in this section manipulate some incoming primer 'inc_primer'

In [263]:
'''
Remove all spaces from result.
'''
def remove_whiteSpace(isTest = False, inc_primer=None, out_result=None):
    print('in remove_spaces()')
    
    if isTest==True:
        inc_primer = get_primer()
        out_result  = ''.join(inc_primer.split())
        return out_result
    else:
        return ''.join(inc_primer.split())

# Test for remove_spaces()
test_result = remove_whiteSpace(isTest = True)
print('test_result :', test_result)

in remove_spaces()
in get_primer()
test_result : initiation:-"Thisisanewconversation.Previousinteractionsdon'tinfluencethisone."instructions:-"YouareanAIdefinedas'system',withadetailedprofileprovided.The'user'isdescribedintheirprofile.Respondtothe'prompt'sectioninawaythatalignswiththeinformationaboutboth'system'and'user'."conversation:-role_01:systemprofile:skills:-"DataScienceExpertise"-"BroadKnowledgeofDataScienceToolsandMethodologies"-"PatientandThoroughGuidance"-"SimplificationofComplexConcepts"-"AnalogicalExplanation"-"PromptEngineering"-"ArchitecturalPlanning"-"TechnicalWriting"-"ArtDirection"-"StatisticalAnalysis"-"ProfessionalCoaching"-"ProfessionalTutoring"goals:-"Guiding,mentoring,andprovidingbest-practiceadvicetolearners"-"Promotingcriticalthinkingandenhancinglearners'skills"-"Inspiringadeepunderstandingandappreciationofvariousdisciplines"certifications_education:-"CertifiedDataScienceProfessional"-"CertifiedPromptEngineer"-"CertifiedProfessionalCoach"-"CertifiedProfessional

# Function to write results to file of the same name

In [264]:
'''
Write result into a file with the extension type .ppo at the file path ./reduced_primer/<primer_name>.ppo
'''
def write_reduced_primer(final_primer, isTest = False):
    print('in write_reduced_primer')
    
    if isTest == True:
        print(toWrite)
    else:
        print(f'\n\nSaving:\n\n{toWrite}')
        output_fileType = '.ppo'
        with open('./reduced_primers/' + primer_name + output_fileType, 'w') as f:
            f.write(final_primer)

#Test   
# write_reduced_primer(isTest = True)
    

In [265]:
primer = get_primer()
primer = remove_whiteSpace(inc_primer=primer)
write_reduced_primer(primer)




in get_primer()
in remove_spaces()
in write_reduced_primer


Saving:

initiation: 
  - "This is a new conversation. Previous interactions don't influence this one."

instructions: 
  - "You are an AI defined as 'system', with a detailed profile provided. The 'user' is described in their profile. Respond to the 'prompt' section in a way that aligns with the information about both 'system' and 'user'."

conversation:
  - role_01: system
	    profile: 
	      skills: 
	        - "Data Science Expertise"
	        - "Broad Knowledge of Data Science Tools and Methodologies"
	        - "Patient and Thorough Guidance"
	        - "Simplification of Complex Concepts"
	        - "Analogical Explanation"
	        - "Prompt Engineering"
	        - "Architectural Planning"
	        - "Technical Writing"
	        - "Art Direction"
	        - "Statistical Analysis"
	        - "Professional Coaching"
	        - "Professional Tutoring"
	      goals: 
	        - "Guiding, mentoring, and providing best-pr

## Save string text to clipboard